In [1]:
import os

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [25]:
DEPTH_CONSTRAINT = 2

In [ ]:
from simpletransformers.classification import (
    MultiLabelClassificationModel, MultiLabelClassificationArgs
)
import pandas as pd
import logging
import numpy as np



transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# %%
import dvc.api

df_test = pd.read_json(dvc.api.get_url(
    'datasets/data/query_label/processed/Offshore_Labelled_Query_Classification_Test_V2.json',
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
), lines=True)

# %%
df_tax = pd.read_json(dvc.api.get_url(
    'datasets/data/taxonomy/wish_newtax.json',
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
), lines=True)
df_tax = df_tax[(df_tax.category_path.apply(len) > 0) & (df_tax.category_path.apply(lambda x: len(x.split(' > ')) <= DEPTH_CONSTRAINT))]

# %%
LABEL_SET = sorted(df_tax.category_path.str.lower().str.strip().tolist()) + ['unknown']

# %%
LABEL_NAME_TO_ID = {i: ind for ind, i in enumerate(LABEL_SET)}
# %%

# %%
def categories2labels(cats):
    if len(cats) == 0:
        cats = ['unknown']
    labs = [0] * len(LABEL_NAME_TO_ID)
    for c in cats:
        labs[LABEL_NAME_TO_ID[c]] = 1
    return labs


In [4]:
model = MultiLabelClassificationModel(
    "xlmroberta",
    "outputs_stage3/best_model",
)

In [10]:
import torch

In [15]:
df_test['labels'] = df_test['query_classification_lists'].apply(
    lambda x: categories2labels([' > '.join(i.lower().strip().split(' > ')[:DEPTH_CONSTRAINT]) for i in x]))
df_test['text'] = df_test['query']

In [16]:
result, model_outputs, wrong_predictions = model.eval_model(
    df_test
)

Running Evaluation:   0%|          | 0/1681 [00:00<?, ?it/s]

In [17]:
result

{'LRAP': 0.6229397274199611, 'eval_loss': 0.017392489964675416}

In [18]:
from sklearn.metrics import classification_report

In [20]:
print(classification_report(np.array(df_test['labels'].tolist()), model_outputs>.5, zero_division=0,))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.67      0.04      0.08        45
           2       0.00      0.00      0.00        24
           3       0.25      0.07      0.11        42
           4       0.00      0.00      0.00        51
           5       0.68      0.71      0.69        24
           6       0.69      0.78      0.73        72
           7       0.33      0.57      0.42         7
           8       0.75      0.61      0.67        67
           9       0.00      0.00      0.00         5
          10       0.67      0.67      0.67         3
          11       0.35      0.47      0.40        49
          12       0.58      0.79      0.67        14
          13       0.53      0.34      0.41        53
          14       0.33      0.50      0.40         8
          15       0.76      0.52      0.62        25
          16       0.82      0.38      0.51        48
          17       0.83    

In [21]:
import pandas as pd

In [21]:
df_qcv2 = pd.read_json(
    "../../models/multitask_multimodal_multilingual/version_9/clm-epoch=1-step=2600--wish_labelled_query_offshore_test_V2--test.json", 
    lines=True)

In [22]:
df_qcv2 = df_qcv2.sort_values(['batch_indices', 'rank_indices']).groupby('batch_indices').agg({
    'prediction_decoded': lambda x: [i for i in x], 
    'prob': lambda x: [i for i in x]
})

In [23]:
recs = []
for i in df_qcv2.to_dict('records'):
    prediction_decoded_confident = []
    for pred, prob in zip(i['prediction_decoded'], i['prob']):
        if prob >= .05:
            prediction_decoded_confident.append(pred)
    i['prediction_decoded_confident'] = prediction_decoded_confident
    recs.append(i)
df_qcv2 = pd.DataFrame(recs)

In [26]:
df_qcv2['labels'] = df_qcv2['prediction_decoded_confident'].apply(
    lambda x: categories2labels([' > '.join(i.lower().strip().split(' > ')[:DEPTH_CONSTRAINT]) for i in x]))

In [27]:
print(classification_report(np.array(df_test['labels'].tolist()), np.array(df_qcv2['labels'].tolist())))

/opt/conda/envs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.73      0.24      0.37        45
           2       0.46      0.46      0.46        24
           3       0.35      0.29      0.32        42
           4       0.36      0.76      0.49        51
           5       0.56      0.83      0.67        24
           6       0.76      0.76      0.76        72
           7       0.39      1.00      0.56         7
           8       0.78      0.73      0.75        67
           9       0.50      0.20      0.29         5
          10       0.67      0.67      0.67         3
          11       0.35      0.86      0.50        49
          12       0.43      0.93      0.59        14
          13       0.54      0.81      0.65        53
          14       0.36      0.62      0.45         8
          15       0.50      0.76      0.60        25
          16       0.59      0.73      0.65        48
          17       0.81    